<h1> Epitope Prediction </h1>

This tutorial illustrates the use of epytope to predict T cell receptor (TCR) binding to epitopes and how to analyze results. epytope offers a long list of epitope prediction methods and was designed in such a way that extending epytope with your favorite method is easy.

This tutorial will entail:
- Simple TCR-epitope prediction from a repertoire of TCRs and selected epitopes
- Manipulation of the results
- Consensus prediction with multiple prediction methods
- Integration of a new prediction method


<h2> Chapter 1: The basics </h2>
<br/>
We first start with importing the needed packages.

In [1]:
import sys
sys.path.append('../..')

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from epytope.Core import Peptide, Allele, TCREpitope, ImmuneReceptorChain, ImmuneReceptor

from epytope.IO import IRDatasetAdapterFactory
from epytope.TCRSpecificityPrediction import TCRSpecificityPredictorFactory

Lets start of with something simple: Defining TCR-epitopes (consisting of peptides and optionally HLA alleles) and TCRs. You find all basic classes under `epytope.Core`. For more information on HLA alleles and peptides see the tutorial `EpitopePrediction.ipynb`.

In [3]:
peptide = Peptide("SYFPEITHI")
allele = Allele("HLA-A*02:01")
epitope_1 = TCREpitope(peptide=peptide, allele=allele)
epitope_1

TCR EPITOPE:
 PEPTIDE SYFPEITHI
 bound by ALLELE: HLA-A*02:01

In [4]:
epitope_2 = TCREpitope(peptide="EAAGIGILTV", allele=None)
epitope_2

TCR EPITOPE:
 PEPTIDE EAAGIGILTV

Next we will start to define a TCR from the superclass Immune receptor, which also can store B cell receptors (BCRs). Each TCR consists of at least 1 Immune Receptor Chain, that is typicall an α- or β-chain for TCR-Epitope prediction, as γ-δ TCRs are typically not considered by the predictors. Additionally, you can provide the celltype (e.g. "CD4 T cell"), and its host organism.

Each IR-Chain can contain the following information: chain type (out of ["TRA", "TRB", "TRG", "TRD", "IGK", "IGH", "IGL"]), V-, (D-), and J-gene, and the complementary determining regions 1-3 (CDR1-3). Note, that all information is optional except the CDR3 sequence.

In [5]:
alpha_1 = ImmuneReceptorChain(chain_type="TRA", 
                              v_gene="TRAV26-1", 
                              j_gene="TRAJ43",
                              cdr3="CIVRAPGRADMRF")
beta_1 = ImmuneReceptorChain(chain_type="TRB", 
                             v_gene="TRBV13", 
                             j_gene="TRBJ1-5",
                             cdr3="CASSYLPGQGDHYSNQPQHF")
tcr_1 = ImmuneReceptor([alpha_1, beta_1], cell_type='CD4 T cell', organism=None)

beta_2 = ImmuneReceptorChain(chain_type="TRB", 
                             v_gene="TRBV5-8", 
                             j_gene="TRBJ1-1",
                             cdr3="CASSTIRQGSNTFF")
tcr_2 = ImmuneReceptor([beta_2], cell_type='T cell')

tcrs = [tcr_1, tcr_2]

Of course we don't have to specify a full TCR repertoire by hand. We implemented interfaces to several formats of IRs (AIRR, Scirpy), databases (IEDB, VDJdb, McPas-TCR), and generall data frames.

In [6]:
path_data = '../../scrubs/vdjdb.tsv'

In [7]:
for name, version in IRDatasetAdapterFactory.available_methods().items():
    print(name, ",".join(version))

vdjdb 2023.06.01
iedb 1.0.0
mcpas-tcr 1.0.0
scirpy 0.10.1
airr scirpy:0.10.1
dataframe 0.0.0.1


In [8]:
tcr_repertoire = IRDatasetAdapterFactory("vdjdb")
tcr_repertoire.from_path(path_data)
tcr_repertoire.receptors = tcr_repertoire.receptors[:20]

In [9]:
for tcr in tcr_repertoire.receptors[:3]:
    print(tcr)
    print()

ALPHA-BETA T CELL RECEPTOR
in HomoSapiens
 IMMUNE RECEPTOR CHAIN: TRA
 CDR3: CIVRAPGRADMRF
 V_gene: TRAV26-1*01
 J_gene: TRAJ43*01
 IMMUNE RECEPTOR CHAIN: TRB
 CDR3: CASSYLPGQGDHYSNQPQHF
 V_gene: TRBV13*01
 J_gene: TRBJ1-5*01

ALPHA-BETA T CELL RECEPTOR
in HomoSapiens
 IMMUNE RECEPTOR CHAIN: TRA
 CDR3: CAVPSGAGSYQLTF
 V_gene: TRAV20*01
 J_gene: TRAJ28*01
 IMMUNE RECEPTOR CHAIN: TRB
 CDR3: CASSFEPGQGFYSNQPQHF
 V_gene: TRBV13*01
 J_gene: TRBJ1-5*01

ALPHA-BETA T CELL RECEPTOR
in HomoSapiens
 IMMUNE RECEPTOR CHAIN: TRA
 CDR3: CAVKASGSRLT
 V_gene: TRAV2*01
 IMMUNE RECEPTOR CHAIN: TRB
 CDR3: CASSYEPGQVSHYSNQPQHF
 V_gene: TRBV13*01
 J_gene: TRBJ1-5*01



We can also convert the resulting repertoire back into a data frame if common format for better readability.

In [10]:
df_tmp = tcr_repertoire.to_pandas()
df_tmp.head()

,celltype,organism,VDJ_chain_type,VDJ_cdr3,VDJ_v_gene,VDJ_d_gene,VDJ_j_gene,VJ_chain_type,VJ_cdr3,VJ_v_gene,VJ_j_gene
0,alpha-beta T cell,HomoSapiens,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,,TRBJ1-5*01,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01
1,alpha-beta T cell,HomoSapiens,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,,TRBJ1-5*01,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01
2,alpha-beta T cell,HomoSapiens,TRB,CASSYEPGQVSHYSNQPQHF,TRBV13*01,,TRBJ1-5*01,TRA,CAVKASGSRLT,TRAV2*01,
3,alpha-beta T cell,HomoSapiens,TRB,CASSALASLNEQFF,TRBV14*01,,TRBJ2-1*01,TRA,CAYRPPGTYKYIF,TRAV38-2/DV8*01,TRAJ40*01
4,alpha-beta T cell,HomoSapiens,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,,TRBJ1-5*01,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01


epytope has only one entry point to the different prediction methods, namely `TCRSpecificityPredictorFactory`. It handles the initialization of the different methods and also collects newly implemented prediction methods if properly implemented. To see which prediction methods epytope supports `TCRSpecificityPredictorFactory` can helps here as well:

In [11]:
for name, version in TCRSpecificityPredictorFactory.available_methods().items():
    print(name, ",".join(version))

imrex 
titan 1.0.0
ergo-ii 
pmtnet 
epitcr 
atm-tcr 


In [12]:
epitope_1

TCR EPITOPE:
 PEPTIDE SYFPEITHI
 bound by ALLELE: HLA-A*02:01

In [20]:
len(tcr_repertoire.receptors)

20

In [ ]:
epitope_1

In [29]:
predictor = TCRSpecificityPredictorFactory("titan")
results = predictor.predict(tcr_repertoire, [epitope_1] * len(tcr_repertoire.receptors), 
                            conda="epytope_torch10",
                            pairwise=False)
results.head(3)

['celltype', 'organism', 'VDJ_chain_type', 'VDJ_cdr3', 'VDJ_v_gene', 'VDJ_d_gene', 'VDJ_j_gene', 'VJ_chain_type', 'VJ_cdr3', 'VJ_v_gene', 'VJ_j_gene', 'Epitope', 'MHC', 'Score']
[('TCR', 'celltype'), ('TCR', 'organism'), ('TCR', 'VDJ_chain_type'), ('TCR', 'VDJ_cdr3'), ('TCR', 'VDJ_v_gene'), ('TCR', 'VDJ_d_gene'), ('TCR', 'VDJ_j_gene'), ('TCR', 'VJ_chain_type'), ('TCR', 'VJ_cdr3'), ('TCR', 'VJ_v_gene'), ('TCR', 'VJ_j_gene'), ('Epitope', 'Peptide'), ('Epitope', 'MHC'), ('Method', 'TITAN')]


TCR                                                    \
            celltype     organism VDJ_chain_type              VDJ_cdr3   
0  alpha-beta T cell  HomoSapiens            TRB  CASSYLPGQGDHYSNQPQHF   
1  alpha-beta T cell  HomoSapiens            TRB   CASSFEPGQGFYSNQPQHF   
2  alpha-beta T cell  HomoSapiens            TRB  CASSYEPGQVSHYSNQPQHF   

                                                                   \
  VDJ_v_gene VDJ_d_gene  VDJ_j_gene VJ_chain_type         VJ_cdr3   
0  TRBV13*01             TRBJ1-5*01           TRA   CIVRAPGRADMRF   
1  TRBV13*01             TRBJ1-5*01           TRA  CAVPSGAGSYQLTF   
2  TRBV13*01             TRBJ1-5*01           TRA     CAVKASGSRLT   

                             Epitope                 Method  
     VJ_v_gene  VJ_j_gene    Peptide          MHC     TITAN  
0  TRAV26-1*01  TRAJ43*01  SYFPEITHI  HLA-A*02:01  0.177025  
1    TRAV20*01  TRAJ28*01  SYFPEITHI  HLA-A*02:01  0.190090  
2     TRAV2*01             SYFPEITHI  HLA-A*02:01  0.169054

In [35]:
predictor = TCRSpecificityPredictorFactory("titan")
results = predictor.predict(tcr_repertoire, [epitope_1, epitope_2], 
                            conda="epytope_torch10",
                            pairwise=True)
results.head(3)

   VDJ_v_gene  VDJ_j_gene              VDJ_cdr3     Epitope     Score
0   TRBV13*01  TRBJ1-5*01  CASSYLPGQGDHYSNQPQHF   SYFPEITHI  0.177025
1   TRBV13*01  TRBJ1-5*01   CASSFEPGQGFYSNQPQHF   SYFPEITHI  0.190090
2   TRBV13*01  TRBJ1-5*01  CASSYEPGQVSHYSNQPQHF   SYFPEITHI  0.169054
3   TRBV14*01  TRBJ2-1*01        CASSALASLNEQFF   SYFPEITHI  0.342878
4   TRBV28*01  TRBJ2-1*01         CASSFTPYNEQFF   SYFPEITHI  0.192828
5   TRBV28*01  TRBJ1-1*01        CASSPQGLGTEAFF   SYFPEITHI  0.347369
6   TRBV28*01  TRBJ1-5*01        CAEGQGFVGQPQHF   SYFPEITHI  0.316065
7   TRBV28*01  TRBJ2-3*01       CASLRSAVWADTQYF   SYFPEITHI  0.290837
8   TRBV28*01  TRBJ1-5*01        CASTLTGLGQPQHF   SYFPEITHI  0.340190
9   TRBV28*01  TRBJ1-1*01        CASSFQGLGTEAFF   SYFPEITHI  0.192499
10  TRBV27*01  TRBJ2-1*01           CASSFNDEQFF   SYFPEITHI  0.221847
11  TRBV13*01  TRBJ1-5*01  CASSYLPGQGDHYSNQPQHF  EAAGIGILTV  0.449186
12  TRBV13*01  TRBJ1-5*01   CASSFEPGQGFYSNQPQHF  EAAGIGILTV  0.203096
13  TRBV13*01  TRBJ1

TCR                                                    \
            celltype     organism VDJ_chain_type              VDJ_cdr3   
0  alpha-beta T cell  HomoSapiens            TRB  CASSYLPGQGDHYSNQPQHF   
1  alpha-beta T cell  HomoSapiens            TRB   CASSFEPGQGFYSNQPQHF   
2  alpha-beta T cell  HomoSapiens            TRB  CASSYEPGQVSHYSNQPQHF   

                                                                   \
  VDJ_v_gene VDJ_d_gene  VDJ_j_gene VJ_chain_type         VJ_cdr3   
0  TRBV13*01             TRBJ1-5*01           TRA   CIVRAPGRADMRF   
1  TRBV13*01             TRBJ1-5*01           TRA  CAVPSGAGSYQLTF   
2  TRBV13*01             TRBJ1-5*01           TRA     CAVKASGSRLT   

                          SYFPEITHI, HLA-A*02:01 EAAGIGILTV  
     VJ_v_gene  VJ_j_gene                  TITAN      TITAN  
0  TRAV26-1*01  TRAJ43*01               0.177025   0.449186  
1    TRAV20*01  TRAJ28*01               0.190090   0.203096  
2     TRAV2*01                          0.169054   0.402543

Lets select one and do predictions. 

In [ ]:
predictor = TCRSpecificityPredictorFactory("imrex")
results = predictor.predict(tcr_repertoire, [epitope_1] * len(tcr_repertoire.receptors), 
                            conda="tcr_predictors",
                            pairwise=False)
results.head(3)

In [ ]:
predictor = TCRSpecificityPredictorFactory("imrex")
results = predictor.predict(tcr_repertoire, [epitope_1, epitope_2], 
                            conda="tcr_predictors")
results.head(3)

In [ ]:
predictor = TCRSpecificityPredictorFactory("ergo-ii")
results = predictor.predict(tcr_repertoire, [epitope_1, epitope_2], 
                            repository="../../external/ERGO-II")
results.head(3)

In [ ]:
raise ValueError

External tools like `NetMHC` offer two additional flags when calling `.predict()`, `command="/path/to/binary"` and `options="command options"`. `command=""` specifies the path to an alternative binary that should be used instead of the one that is globally registered. With `options=""` you can specify additional commands that will directly be passed to the command line call without any sanity checks.

In [ ]:
predictor = EpitopePredictorFactory("Syfpeithi", version="1.0")
results = predictor.predict(peptides2, alleles=alleles, options="-s -b") 
results.head()

<h2> Chapter 2: Data manipulation and consensus prediction</h2>
<br/>
The predictor all return a data table like object (DataFrame) storing the peptide and allele objects, as well as the predicted results. Because it is a inherited pandas DataFrame we can do all sorts of nifty thinks. 

For example exporting the results to tsv:

In [ ]:
results.to_csv('./data/test.tsv', sep='\t')

Or we could plot the score distribution of a particular HLA allele.

In [ ]:
results.hist()

To combine prediction results we can use `merge_results` from `epytope.Core`. In addition to the result object we want to merge, also have to specify the type of these objects (here `EpitopePredictionResult`). The function will return a merged results object of the same type.

In [ ]:
results = [EpitopePredictorFactory(m).predict(peptides,alleles=alleles) 
                    for m in ["Syfpeithi","BIMAS","SMM"]]
df = results[0].merge_results(results[1:])
df

We also can filter the predicted epitopes based on their prediction values with the function `filter_result` from `epytope.Core`.

In [ ]:
#you can either use pre-defined operators from `operator`
from operator import ge
#or define you own comparator function like this
comparator = lambda a,b: a > b

df.filter_result(['HLA-A*02:01', 'syfpeithi', 'Score'], comparator, 20.0)

With that one can combine several prediction tools to form a consensus prediction method.<br/><br/>
<h2> Chapter 3: Implementation of a new epitope prediction method </h2>
<br/>
epytope possesses a potent plugin system allowing the user to extend its capability quite easily. To include a new epitope prediction method one simply has to inherit from `epytope.Core.AEpitopePrediction` and implement its interface. For methods calling an external prediction tool additionally have to inherit from `epytope.Core.AExternal`. SVM based methods also define a specific interface via `epytope.Core.ASVM`. epytope uses SVMlight and its python binding svmlight 0.4.

If you want to be very specific and fully integrate your method in all of epytope's capabilities please use one of the three major interfaces `APSSMEpitopePrediction`, `ASVMEpitopePrediction`, or `AExternalEpitopePrediction` from `epytope.EpitopePrediction`.

In [ ]:
from epytope.EpitopePrediction import APSSMEpitopePrediction
from epytope.Core import EpitopePredictionResult
import random
import pandas

class RandomTCRSpecificityPrediction(APSSMEpitopePrediction):
    __tcr_length = (0, 40)
    __epitope_length = (0, 40)
    __name = "random"
    __version= "1.0"
    
    #the interface defines three class properties
    @property
    def name(self):
        #returns the name of the predictor
        return self.__name
    
    @property
    def tcr_length(self):
        #returns the supported tcr lengths (min, max)
        return self.__supported_tcr_length
    
    @property
    def epitope_length(self):
        #returns the supported epitope lengths (min, max)
        return self.__supported_epitope_length
    
    @property
    def version(self):
        #returns the version of the predictor
        return self.__version
    
    #additionally the interface defines a function `predict` 
    #that consumes a list of peptides or a single peptide and optionally a list 
    #of allele objects
    #
    #this method implements the complete prediction routine
    def predict(self, tcrs, epitopes, pairwise=True, **kwargs):
        
        #test whether one peptide or a list
        if isinstance(peptides, str):
            peptides = list(peptides)
        
        #if no alleles are specified do predictions for all supported alleles
        if alleles is None:
            alleles = self.supportedAlleles
        else:
            #filter for supported alleles
            alleles = filter(lambda a: a.name in self.supportedAlleles, alleles) 
        
        scores = {}
        #now predict binding/non-binding for each peptide at random
        for a in alleles:
            scores[a] = {}
            for p in peptides:
                if random.random() >= 0.5:
                    scores[a][p] = 1.0
                else:
                    scores[a][p] = 0.0
        
        #create EpitopePredictionResult object from nested dictionary. This is a multi-column DataFrame 
        #with allele, method and scoretype as columns and peptides as indices
        result = {allele: {self.__scoretype: list(scores.values())[i]} for i, allele in enumerate(alleles)}
        
        # converts the nested dictionary to a EpitopePredictionResult
        df_result = EpitopePredictionResult.from_dict(result, peptides, self.__name)
        
        return df_result


Now lets use our new predictor.

In [ ]:
TCRSpecificityPredictorFactory("random").predict(peptides)

The predictor is now fully integrated and can be used in any context defined by epytope.